# Récupération d'un jeu de données de binance

In [1]:
import requests
from datetime import datetime, date, timedelta

timestamp = int(datetime.strptime("2022-11-14T00:00:00", "%Y-%m-%dT%H:%M:%S").strftime("%s"))*1000

data = {"symbol": "BTCEUR", "interval": "1s", "startTime":timestamp, "limit": 675}
url = 'https://api.binance.com/api/v3/klines'
binance_response = requests.get(url, params=data)


# Préparation d'un dataframe avec 86 400 lignes en vue d'écritures des tables

In [2]:
import pandas as pd

df = pd.DataFrame(binance_response.json())
columns = ['timestamp', 'open', 'close', 'high', 'low', 'quantity']
df = df.drop(list(range(6,df.shape[1])), axis=1) \
    .astype("float") \
    .rename(columns={i: c for i, c in enumerate(columns)})

df["timestamp"] = df["timestamp"].floordiv(1000).astype("int")

for _ in range(0,7):
    df2 = df.copy()
    df2["timestamp"] = df2["timestamp"].add(df.iloc[0]["timestamp"]-df.iloc[-1]["timestamp"]-1).astype("int")
    df = pd.concat([df2, df])


((86400, 6),
 timestamp      int64
 open         float64
 close        float64
 high         float64
 low          float64
 quantity     float64
 dtype: object,
 1668294675.0,
 1668381074.0)

# Création de la base SQLite avec SQLAlchemy

In [3]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, Float, MetaData, create_engine

engine = create_engine('sqlite:///many-tables.db', echo=False)
meta = MetaData()


# Création des 1000 tables et insertion des données

In [4]:
from datetime import datetime, date

df_sql = df.copy()
for _ in range(0,1000):
    day = date.fromtimestamp(df_sql.iloc[0]["timestamp"]).strftime("%Y_%m_%d")
    table_name = f"SYMBOL_{day}"
    engine.execute(f"DROP TABLE IF EXISTS {table_name}")
    symbol = Table(f"{table_name}", meta,
                    Column('timestamp', Integer, nullable=False),
                    Column('open', Float, nullable=False),
                    Column('close', Float, nullable=False),
                    Column('high', Float, nullable=False),
                    Column('low', Float, nullable=False),
                    Column('quantity', Float, nullable=False),
                    extend_existing=True
                )
    meta.create_all(engine)

    df_sql.to_sql(f"{table_name}",con = engine, index=False, if_exists='append')
    df_sql["timestamp"] = df_sql["timestamp"].add(df_sql.iloc[0]["timestamp"]-df_sql.iloc[-1]["timestamp"]-1).astype("int")



# Exemple de requête

In [5]:
start = datetime.now().replace(hour=12)-timedelta(days = 200)
end = start + timedelta(seconds = 1000)

start = int(start.strftime("%s"))
end = int(end.strftime("%s"))

day = date.fromtimestamp(start).strftime("%Y_%m_%d")

conn = engine.connect()
cursor = conn.execute(f"SELECT * FROM SYMBOL_{day} WHERE timestamp >= {start:.0f} AND timestamp < {end:.0f}")

result = list(cursor)
len(result)


1000